## 1 Загрузка данных

In [8]:
import random

random.seed(42)

In [12]:
filename = "goods_pool.npy'

if os.path.exists(filename):
    print("File exist")
    goods_pool = np.load(filename)

File exist


In [14]:
#https://stackoverflow.com/questions/47117569/how-to-get-word2index-from-gensim
'''
word = "whatever"  # for any word in model
i = model.vocab[word].index
model.index2word[i] == word  # will be true
'''

goods_vocab = {}
goods_index = {}
counter = 0
goods_pool_indexed = []
gvec = []
goods_reverse_index = []

for i in goods_pool:
    for w in i:
        if w not in goods_vocab:
            goods_vocab[w] = model1.index_to_key[w]
            goods_index[w] = counter
            goods_reverse_index.append(w) # для этапа расширения newquads
            counter += 1
            gvec.append(model1[w]) # для этапа обучения после расширения newquads

    goods_pool_indexed.append([goods_index[i[0]],
                               goods_index[i[1]],
                               goods_index[i[2]],
                               goods_index[i[3]]                               
                               ])

len(goods_vocab) #len(qvec)

272746

## 2 Построение представлений для набора четвёрок

In [24]:
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import copy

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [25]:
class PletNet(nn.Module):
    def __init__(self, p):
        super().__init__()
        self.linear = nn.Linear(p, p)#, bias=False)
    
    def forward(self, v):
        return self.linear(v)
    
    def inference(self, v):
        with torch.no_grad():
            #return self.linear(torch.from_numpy(v).float())
            return self.linear(v)

In [26]:
class QuadrupletLoss(nn.Module):
    def __init__(self, margin):
        super().__init__()
        self.margin = margin
        self.cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    
    def forward(self, batch_i, batch_j, batch_k, batch_l, size_average=True):
        d_ij = 1 - self.cos(batch_i, batch_j)
        d_kl = 1 - self.cos(batch_k, batch_l)
        losses = F.relu(d_ij - d_kl + self.margin)
        
        res = losses.sum() # можно ли сделать лосс число положительных разностей (дискретный лосс) ?
        
        #losses[losses > 0] = 1
        
        #print("loss int: ", int(losses.sum()))
        return res #losses.mean() if size_average else losses.sum()

In [27]:
#есть встроенный в torch(может работать быстрее)

class TripletLoss(nn.Module):
    def __init__(self, margin):
        super().__init__()
        self.margin = margin
        self.cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    
    def forward(self, batch_i, batch_j, batch_k, batch_l, size_average=True):
        d_ij = 1 - self.cos(batch_i, batch_j)
        d_kl = 1 - self.cos(batch_k, batch_l)
        losses = F.relu(d_ij - d_kl + self.margin)
        
        res = losses.sum()
                
        return res

In [28]:
def hand_checker(vec, gquads, on_gpu = False):

    a,b,c,d = list(zip(*gquads))
    
    if on_gpu:
        batch_i = torch.index_select(vec, 0, torch.tensor(a).to(device))
        batch_j = torch.index_select(vec, 0, torch.tensor(b).to(device))
        batch_k = torch.index_select(vec, 0, torch.tensor(c).to(device))
        batch_l = torch.index_select(vec, 0, torch.tensor(d).to(device))
    else:
        batch_i = torch.index_select(vec, 0, torch.tensor(a))
        batch_j = torch.index_select(vec, 0, torch.tensor(b))
        batch_k = torch.index_select(vec, 0, torch.tensor(c))
        batch_l = torch.index_select(vec, 0, torch.tensor(d))
    
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    d_ij = 1 - cos(batch_i, batch_j)
    d_kl = 1 - cos(batch_k, batch_l)

    diff = d_ij - d_kl
    diff[diff > 0] = 0
    diff[diff < 0] = 1

    counter = torch.sum(diff)
    #print(counter)
    return int(counter)#float(counter / len(gquads))

In [29]:
def fit_plet(num_epochs, model, loss_fn, opt, vec, a, b, c, d, batches = False):
    
    for epoch in tqdm.tqdm(range(num_epochs)):
    
        pred = model(vec)
        
        if batches:
            ind = torch.tensor(random.sample(range(500000), 100000)).to(device)
            aa = torch.index_select(a, 0, ind)
            bb = torch.index_select(b, 0, ind)
            cc = torch.index_select(c, 0, ind)
            dd = torch.index_select(d, 0, ind)
        else:
            aa = a
            bb = b
            cc = c
            dd = d
             
        loss = loss_fn(
                torch.index_select(pred, 0, aa),
                torch.index_select(pred, 0, bb),
                torch.index_select(pred, 0, cc),
                torch.index_select(pred, 0, dd)
               )
        #print(loss)        
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
        #if epoch % 200 == 0:
        #    for g in opt.param_groups:
        #        g['lr'] *= 0.1
        
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    return model.inference(vec.to(device)).cpu()###new_vec

In [30]:
model = PletNet(300).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
quadruplet_loss = TripletLoss(0.1)

In [31]:
vec = torch.empty(len(goods_vocab), 300)
nn.init.xavier_normal_(vec)
hand_checker(vec, goods_pool_indexed)

249550

In [32]:
a,b,c,d = list(zip(*goods_pool_indexed))
#a,b,c,d = list(a), list(b), list(c), list(d)

In [33]:
# попытки решить проблему медленной сходимости на 150000
# больше эпох
# меньше lr
# батчи
# другая инициализация(xavier)

In [34]:
for g in opt.param_groups:
    print(g['lr'])

0.001


In [35]:
fit_plet(300, model, quadruplet_loss, opt, vec.to(device), #gquads
               torch.tensor(a).to(device),
               torch.tensor(b).to(device),
               torch.tensor(c).to(device),
               torch.tensor(d).to(device),
               #batches = True
          )

  1%|          | 2/300 [00:00<00:27, 10.65it/s]

Epoch [1/300], Loss: 49999.7734
Epoch [2/300], Loss: 49998.0820
Epoch [3/300], Loss: 49996.2422


  2%|▏         | 6/300 [00:00<00:26, 11.04it/s]

Epoch [4/300], Loss: 49994.2188
Epoch [5/300], Loss: 49991.9766
Epoch [6/300], Loss: 49989.4922


  3%|▎         | 8/300 [00:00<00:26, 11.11it/s]

...

 99%|█████████▉| 298/300 [00:26<00:00, 11.24it/s]

Epoch [297/300], Loss: 43901.5859
Epoch [298/300], Loss: 43902.4609
Epoch [299/300], Loss: 43903.4297


100%|██████████| 300/300 [00:26<00:00, 11.20it/s]


Epoch [300/300], Loss: 43903.1016


tensor([[ 1.8630e-03,  1.8988e-03, -1.3293e-03,  ...,  4.3269e-03,
          7.2472e-04,  4.9227e-04],
        [ 6.6027e-03, -5.8370e-03, -1.8313e-03,  ...,  1.1590e-04,
         -5.2817e-04, -7.8654e-04],
        [ 1.9816e-03, -9.5562e-03, -3.7453e-03,  ...,  1.8950e-03,
          3.3220e-03, -8.2199e-04],
        ...,
        [ 3.8793e-03,  4.0349e-03, -3.2841e-03,  ..., -1.8888e-03,
         -2.6456e-03, -2.8035e-03],
        [ 2.0182e-03, -4.5171e-03, -1.0726e-03,  ..., -5.1795e-05,
         -1.2877e-03,  1.7364e-03],
        [ 1.2853e-03, -3.7967e-03,  2.1847e-03,  ...,  2.5730e-03,
          2.9331e-03,  2.7268e-03]])

In [36]:
new_vec = model.inference(vec.to(device)).cpu()

In [37]:
%%time
hand_checker(vec, goods_pool_indexed), hand_checker(new_vec, goods_pool_indexed)

CPU times: user 21.6 s, sys: 35.4 s, total: 57 s
Wall time: 3.07 s


(249550, 295876)

In [38]:
%%time
hand_checker(vec.to(device), goods_pool_indexed, True), hand_checker(new_vec.to(device), goods_pool_indexed, True)

CPU times: user 1.29 s, sys: 32.9 ms, total: 1.33 s
Wall time: 1.32 s


(249550, 295876)

In [42]:
t=0
for quad in goods_pool_indexed:
    a,b,c,d=quad

    d_ij = cosine(new_vec[a], new_vec[b])
    d_kl = cosine(new_vec[c], new_vec[d])

    #if d_ij > d_kl:
        #print(new_vec[a], new_vec[b], new_vec[c], new_vec[d])
    print(a,b,c,d)
    #print(new_vec[a], new_vec[b])
    #print(new_vec[c], new_vec[d])
    print(d_ij, d_kl)
    t+=1
    if t==10:
        break

0 1 0 2
0.5745091736316681 0.5163394808769226
3 4 3 5
0.4406927227973938 0.5397063493728638
6 7 6 8
0.6040153205394745 0.4939696788787842
9 10 9 11
0.47912460565567017 0.5334681272506714
12 13 12 14
0.5495234131813049 0.6191833913326263
15 16 15 17
0.642069935798645 0.5361121892929077
18 19 18 20
0.5824757218360901 0.5085082948207855
21 22 21 23
0.46818089485168457 0.5100868344306946
24 25 24 26
0.49467164278030396 0.5781603455543518
27 28 27 29
0.589226245880127 0.495181679725647
